In [1]:
#Notebook for development of the game
# Imports main modules - Player, MapGenerator, etc.
# Implements pygame controls and GUI

In [1]:
import sys, random, os, math
import numpy as np

#where non-dev code is located
sys.path.append('../app')

In [2]:
#install pygame
#!{sys.executable} -m pip install pygame

In [3]:
import pygame

#config stores constants and default settings
from config import Config
from map_generator import MapGenerator

#helpful pygame wrapper
import pygame_utils as pyg

pygame 2.0.1 (SDL 2.0.14, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
# Game GUI consists of:
# - menu through which the user chooses a few settings the starts the game
# - game screen consisting of
#   - the map and players
#   - some type of status bar on the bottom or side allowing restart and showing info
# - winner outcome screen which displays at game end for a while before going back to the menu

In [ ]:
#incremental building of pygame components since they're sensitive and hard to debug

In [4]:
class TheMap():
    def __init__(self, config, new_map):
        self.config = config
        self.border_size = self.config.map_border_size
        self.tile_size = self.config.terrain_tile_size
        
        #make the map and get the terrain speeds associated with the map
        if new_map:
            print('Creating new map')
            tile_cols, tile_rows = 40, 30
            pixel_dims = (self.tile_size * tile_cols, self.tile_size * tile_rows)
            self.tile_speeds, self.map_path = MapGenerator(self.config, seed=13).generate_map(pixel_dims, 
                                                                        save_path = self.config.maps_path)
        else:
            print('Using default map')
            self.map_path = self.config.map_default_path
            self.tile_speeds = np.load(self.config.map_default_speed_array)
            
        self.blue_flag_tile = (0,0)
        self.blue_flag_area_tiles = []
        self.red_flag_tile = (0,0)
        self.red_flag_area_tiles = []
        
        self.player_tile = (0,0)
        self.blue_agent_tiles = []
        self.red_agent_tiles = []
        
        
    def set_flag_location(self, team, flag_x, flag_y):
        flag_c, flag_r = self.xy_to_cr(flag_x, flag_y)
        
        if team=='blue':
            self.blue_flag_tile = (flag_r, flag_c)
        else:
            self.red_flag_tile = (flag_r, flag_c)
        
        #get tiles in flag area
        flag_area_border_tiles = ((self.config.flag_area_size//self.tile_size) // 2) - 1
        for r in range(flag_r - flag_area_border_tiles, flag_r + flag_area_border_tiles + 1):
            for c in range(flag_c - flag_area_border_tiles, flag_c + flag_area_border_tiles + 1):
                if team=='blue':
                    self.blue_flag_area_tiles.append((r, c))
                else:
                    self.red_flag_area_tiles.append((r, c))
                    
        print('%s flag is on %s, %s, in area %s' % (team, flag_r, flag_c, 
                str(self.blue_flag_area_tiles) if team=='blue' else str(self.red_flag_area_tiles)))
                    
        
    def get_not_allowed_tiles(self):
        idx = np.where(self.tile_speeds==0)
        
        return list(zip(idx[0].tolist(), idx[1].tolist()))
    
    
    def xy_to_cr(self, x, y):
        return math.floor(x // self.config.terrain_tile_size), math.floor(y // self.config.terrain_tile_size)
        
        
    def get_speed(self, x, y):
        tile_col, tile_row = self.xy_to_cr(x, y)
        
        return int(self.tile_speeds[tile_row, tile_col])

In [5]:
class Player():
    def __init__(self, x, y, team, the_map, config):
        '''
        x,y - the player/agent sprite starting location on the map
        team - blue or red
        the_map - a reference to the global map with speeds, flag locations, player locations
        config - configurable settings
        '''
        self.config = config
        
        #w,a,d,s are the keyboard directions for up, left, right, down
        #grab/tag/revive - these might just happen automatically when touching a certain sprite
        self.actions = ['w', 'a', 'd', 's', 'grab', 'tag', 'revive']
        
        #for debugging
        self.speed_terr = {v:k for k,v in self.config.terrain_speeds.items()}
        
        #configured values
        self.max_speed = self.config.player_max_speed
        self.max_energy = self.config.player_max_energy
        self.min_energy = self.config.player_min_energy
        
        self.team = team
        self.the_map = the_map
        self.sprite = None
        
        #current status
        self.speed = self.max_speed
        self.energy = self.max_energy
        self.has_flag = False
        self.is_incapacitated = False
        self.incapacitated_countdown = 100
        self.in_enemy_territory = False
        
        self.x = x
        self.y = y
        self.tile_col, self.tile_row = self.the_map.xy_to_cr(x, y)
        
    
    def update(self, frame):
        #get movement action based on map, players, flag percepts
        action = self.get_action()
        
        #first test to see if the action is legal
        new_x, new_y = self.x, self.y
        
        #regardless of legality of move, animate the sprite
        
        #move right
        if action=='d':
            pyg.changeSpriteImage(self.sprite, 0*8+frame)    
            new_x += self.speed
            
        #move down
        elif action=='s':
            pyg.changeSpriteImage(self.sprite, 1*8+frame)    
            new_y += self.speed
            
        #move left
        elif action=='a':
            pyg.changeSpriteImage(self.sprite, 2*8+frame)    
            new_x -= self.speed
            
        #move up
        elif action=='w':
            pyg.changeSpriteImage(self.sprite, 3*8+frame)
            new_y -= self.speed
            
        #stay still
        else:
            pyg.changeSpriteImage(self.sprite, 1 * 8 + 5)
            
        speed = self.the_map.get_speed(new_x, new_y)
        #determine which tile/grid of the map would this put the player in
        
        if speed:
            #for debugging
            if speed != self.speed:
                print('player moved from %s to %s' % (self.speed_terr[self.speed], self.speed_terr[speed]))
                
            self.speed = speed
            self.x = new_x
            self.y = new_y
            
            pyg.moveSprite(self.sprite, self.x, self.y)
            
        
    def get_action(self):
        pass
    
    
    
class HumanPlayer(Player):
    def __init__(self, x, y, team, the_map, config):
        super().__init__(x, y, team, the_map, config)
        #load sprites
        #there are flag holding, nonflag holding, and incapacitated sprites for blue player (human), blue agent, red agent
        #current sprite may change to flag holding or incapacitated sprite
        self.default_sprite = pyg.makeSprite(self.config.blue_player_sprite_path, 32)
        self.holding_flag_sprite = pyg.makeSprite(self.config.blue_player_with_flag_sprite_path, 32)
        self.incapacitated_sprite = pyg.makeSprite(self.config.blue_player_incapacitated_sprite_path, 8)
                                                   
        self.sprite = self.default_sprite
        
        pyg.moveSprite(self.sprite, x, y, centre=True)
        pyg.showSprite(self.sprite)
        
        
    def get_action(self):
        new_x, new_y = self.x, self.y
        
        #move right
        if pyg.keyPressed("d"):
            return 'd'
            
        #move down
        elif pyg.keyPressed("s"):
            return 's'
            
        #move left
        elif pyg.keyPressed("a"):
            return 'a'
            
        #move up
        elif pyg.keyPressed("w"):
            return 'w'
    
    
    
class AgentPlayer(Player):
    def __init__(self, x, y, team, the_map, config):
        super().__init__(x, y, team, the_map, config)
        self.prev_dir = 'a'
        
        #load sprites
        #there are flag holding, nonflag holding, and incapacitated sprites for blue player (human), blue agent, red agent
        #current sprite may change to flag holding or incapacitated sprite
        if team=='blue':
            self.default_sprite = pyg.makeSprite(self.config.blue_agent_sprite_path, 32)
            self.holding_flag_sprite = pyg.makeSprite(self.config.blue_agent_with_flag_sprite_path, 32)
            self.incapacitated_sprite = pyg.makeSprite(self.config.blue_agent_incapacitated_sprite_path, 8)
        else:
            self.default_sprite = pyg.makeSprite(self.config.red_agent_sprite_path, 32)
            self.holding_flag_sprite = pyg.makeSprite(self.config.red_agent_with_flag_sprite_path, 32)
            self.incapacitated_sprite = pyg.makeSprite(self.config.red_agent_incapacitated_sprite_path, 8)
            
        self.sprite = self.default_sprite
        
        pyg.moveSprite(self.sprite, x, y, centre=True)
        pyg.showSprite(self.sprite)
        
        
    def get_action(self):
        if random.randint(1,20) == 1:
            self.prev_dir = random.choice(['a','w','s','d'])
            
        return self.prev_dir

In [6]:
class CaptureTheFlag():
    def __init__(self, new_map=False):
        '''
        Load background, icon, music, sounds, create flag sprites and areas, create players
        '''
        self.config = Config()
        
        #sounds
        self.grabbed_flag_sound = pyg.makeSound(self.config.grabbed_flag_sound)
        self.incapacitated_sound = pyg.makeSound(self.config.incapacitated_sound)
        self.dropped_flag_sound = pyg.makeSound(self.config.dropped_flag_sound)
        self.tagged_sound = pyg.makeSound(self.config.tagged_sound)
        self.tagged_flag_carrier_sound = pyg.makeSound(self.config.tagged_flag_carrier_sound)
        self.revived_sound = pyg.makeSound(self.config.revived_sound)
        self.victory_sound = pyg.makeSound(self.config.victory_sound)
        
        #for drawing the divide and knowing when someone is off sides
        self.screen_divide_x = (self.config.screen_width//2)
        
        #for debug
        self.speed_terr = {v:k for k,v in self.config.terrain_speeds.items()}
        
        #stores location of terrain speeds including 0 (not allowed areas), flag location, player locations
        self.the_map = TheMap(self.config, new_map)
        
        
        #set screen, title, icon, background, music
        self.__setup(self.the_map.map_path)

        
        #make flags
        self.blue_flag_sprite, self.red_flag_sprite = self.__make_flags()
        
        #update the map object with their locations
        self.the_map.set_flag_location('blue', self.blue_flag_x, self.blue_flag_y)
        self.the_map.set_flag_location('red', self.red_flag_x, self.red_flag_y)
        
        
        #make players with reference to map, and with their locations added to the map
        allowed_blue_sprite_init_tiles = self.__get_allowed_sprite_init_tiles('blue')
    
        #create user player on blue team
        self.user_player = self.__make_player(allowed_blue_sprite_init_tiles, team='blue', agent=False)
        #set location in map
        self.the_map.player_tile = (self.user_player.tile_row, self.user_player.tile_col)
        
        #create other blue players
        self.blue_players = []
        for i in range(self.config.blue_team_size - 1):
            blue_player = self.__make_player(allowed_blue_sprite_init_tiles, team='blue')
            self.blue_players.append(blue_player)
            #add to map
            self.the_map.blue_agent_tiles.append((blue_player.tile_row, blue_player.tile_col))

            
        #get red side non-lake non border tile locations
        allowed_red_sprite_init_tiles = self.__get_allowed_sprite_init_tiles('red')
        
        #create red players
        self.red_players = []
        for i in range(self.config.red_team_size):
            red_player = self.__make_player(allowed_red_sprite_init_tiles, team='red')
            self.red_players.append(red_player)
            self.the_map.red_agent_tiles.append((red_player.tile_row, red_player.tile_col))
            
        #all players
        self.players = [self.user_player] + self.blue_players + self.red_players
        
    
    def run(self):
        flag_in_play = False
        game_winner=''
        
        nextFrame = pyg.clock()
        frame = 0
        
        while True:
            #quit?
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                    
            # We only animate our character every 80ms.
            if pyg.clock() > nextFrame:
                # There are 8 frames of animation in each direction so the modulus 8 allows it to loop                        
                frame = (frame+1)%8                         
                nextFrame += 80 

            #execute each players action
            for player in self.players:
                player.update()
            
            #game logic here - IN PROGRESS
            state = self.__get_state()
            
            #handle states
            if state['blue_wins']:
                pygame.mixer.music.fadeout(3000)
                pyg.pause(3000)
                win_label = pyg.makeLabel('BLUE WINS!!!', 300, self.config.screen_width//2, self.config.screen_height//2, 
                              fontColour='blue', font='Arial', background="black")
                pyg.showLabel(win_label)
                pyg.playSound(self.victory_sound)
                self.end_game_music = pyg.makeMusic(self.config.end_game_music)
                pyg.playMusic()
                pyg.pause(5000)
                pygame.mixer.music.fadeout(3000)
                pyg.pause(3000)
                break
            elif state['red_wins']:
                pygame.mixer.music.fadeout(3000)
                pyg.pause(3000)
                win_label = pyg.makeLabel('RED WINS!!!', 300, self.config.screen_width//2, self.config.screen_height//2, 
                              fontColour='red', font='Arial', background="black")
                pyg.showLabel(win_label)
                pyg.playSound(self.victory_sound)
                self.end_game_music = pyg.makeMusic(self.config.end_game_music)
                pyg.playMusic()
                pyg.pause(5000)
                pygame.mixer.music.fadeout(3000)
                pyg.pause(3000)
                break
           

            pyg.tick(10)

        pyg.endWait()
        
        
    def __get_state(self):
        '''Game logic here.'''
        state = {'blue_wins':False, 'red_wins':False}
        
        #flag grabbed?
        
        #flag dropped
        
        #player tagged
        
        #player revived
        
        #team wins
        
        return state
        
        
    def __setup(self, map_path):
        pyg.screenSize(self.config.screen_width, self.config.screen_height)
        pyg.setIcon(self.config.game_icon_image_path)
        pyg.setWindowTitle(self.config.game_name)
        
        #load the map
        pyg.setBackgroundImage(map_path)
        
        #draw dividing line
        self.__draw_divide()
        
        #music
        pyg.makeMusic(self.config.default_game_music)
        pyg.playMusic(loops=-1)
        #TODO
        #self.single_flag_runner_music = pyg.makeMusic(self.config)
        #self.double_flag_runner_music = pyg.makeMusic(self.config)
        
        
    def __draw_divide(self):
        pyg.drawLine(self.screen_divide_x-3, self.config.map_border_size,
                     self.screen_divide_x-3, self.config.screen_height - self.config.map_border_size, 
                     'blue', linewidth=3)
        
        pyg.drawLine(self.screen_divide_x, self.config.map_border_size, 
                     self.screen_divide_x, self.config.screen_height - self.config.map_border_size, 
                     'red', linewidth=3)
    
    
    def __make_flags(self):
        self.blue_flag_x, self.blue_flag_y = self.__get_flag_position(
            self.the_map, 'blue')
        print('blue flag x %d, y %d, area x %s, area y %d')
        self.red_flag_x, self.red_flag_y = self.__get_flag_position(
            self.the_map, 'red')
        
        self.__draw_flag_areas()
        
        blue_flag_sprite = pyg.makeSprite(self.config.blue_flag_sprite_path)
        pyg.moveSprite(blue_flag_sprite, x=self.blue_flag_x, y=self.blue_flag_y, centre=True)
        pyg.showSprite(blue_flag_sprite)
        
        red_flag_sprite = pyg.makeSprite(self.config.red_flag_sprite_path)
        pyg.moveSprite(red_flag_sprite, x=self.red_flag_x, y=self.red_flag_y, centre=True)
        pyg.showSprite(red_flag_sprite)
        
        return blue_flag_sprite, red_flag_sprite
        
        
    def __get_flag_position(self, the_map, team):
        #choose an available row along a column a set distance from the border
        pad = 20
        
        if team=='blue':
            x = the_map.border_size + self.config.flag_area_size//2 + pad
        else:
            x = self.config.screen_width - the_map.border_size - pad - self.config.flag_area_size//2
            
        flag_tile_c = x//the_map.tile_size
        
        col_speeds = the_map.tile_speeds[:, flag_tile_c]
        idx = np.where(col_speeds > 0)[0].tolist()
        
        flag_tile_r = random.choice(idx)
        y = flag_tile_r * the_map.tile_size
        
        return x, y, x - self.config.flag_area_size//2, y - self.config.flag_area_size//2

    
    def __draw_flag_areas(self):
        pyg.drawEllipse(self.blue_flag_x, self.blue_flag_y, self.config.flag_area_size, self.config.flag_area_size, 
                        colour='blue', linewidth=3)
        
        pyg.drawEllipse(self.red_flag_x, self.red_flag_y, self.config.flag_area_size, self.config.flag_area_size, 
                        colour='red', linewidth=3)
        
        
    def __get_allowed_sprite_init_tiles(self, team):
        side = self.the_map.tile_speeds.shape[1]//3
        if team=='blue':
            idx = np.where(self.the_map.tile_speeds[:,:side]>0)
        else:
            idx = np.where(self.the_map.tile_speeds[:,(side*2):]>0)
            
        allowed_init_tiles = list(zip(idx[0].tolist(), idx[1].tolist()))
        
        random.shuffle(allowed_init_tiles)
        
        return allowed_init_tiles


    def __make_player(self, allowed_sprite_init_tiles, team='', agent=True):
        player_r_idx, player_c_idx = allowed_sprite_init_tiles.pop()

        player_x_pos = player_c_idx * self.config.terrain_tile_size
        player_y_pos = player_r_idx * self.config.terrain_tile_size
        print('%s %s on r=%s, c=%d, x=%d, y=%d' % (team, 'agent' if agent else 'player', 
                                                   player_r_idx, player_c_idx, player_x_pos, player_y_pos))

        if not agent: #this will be a different sprite
            player = HumanPlayer(player_x_pos, player_y_pos, team, self.the_map, self.config)
        else:
            player = AgentPlayer(player_x_pos, player_y_pos, team, self.the_map, self.config)

        return player

In [7]:
pygame.init()
pygame.mixer.init()

game = CaptureTheFlag()

pyg.pause(10000)
pygame.mixer.music.fadeout(3000)
pyg.pause(3000)
pygame.quit()

Using default map
blue flag is on 28, 4, in area [(27, 3), (27, 4), (27, 5), (28, 3), (28, 4), (28, 5), (29, 3), (29, 4), (29, 5)]
red flag is on 6, 37, in area [(5, 36), (5, 37), (5, 38), (6, 36), (6, 37), (6, 38), (7, 36), (7, 37), (7, 38)]
blue player on r=21, c=7, x=140, y=420
blue agent on r=2, c=11, x=220, y=40
blue agent on r=14, c=5, x=100, y=280
blue agent on r=13, c=5, x=100, y=260
blue agent on r=27, c=3, x=60, y=540
red agent on r=5, c=1, x=20, y=100
red agent on r=3, c=8, x=160, y=60
red agent on r=30, c=7, x=140, y=600
red agent on r=11, c=0, x=0, y=220
red agent on r=24, c=12, x=240, y=480


In [11]:
pygame.mixer.music.fadeout(3000)

In [ ]:
pygame.quit()

In [ ]:
### game logic in progress

#everything can be determined by which sprites are touching

for player in players:

    #what is the player touching
    player_touching = allTouching(player.sprite)
    if player_touching:
        for thing in player_touching:
            #how check type?

            #touched flag area and has flag - won
            if player.has_flag and type(thing)==type(blue_flag_area) and player.team=='blue' \
            or type(thing)==type(red_flag_area) and player.team=='red': 
                game_winner=player.team
                break

            #touched a flag
            if type(thing)==type(red_flag) and player.team=='blue' or type(thing)==type(blue_flag) and player.team=='red': 
                #play sound
                playSound(grabbed_flag_sound)

                #remove flag
                if player.team=='blue'
                    killSprite(red_flag)
                else:
                    killSprite(blue_flag)

                #remove player without flag
                killSprite(player.sprite)

                #draw player with flag
                player.sprite = player.has_flag_sprite
                showSprite(player.sprite)

                #if a flag not already in play, play tense music
                if not flag_in_play:
                    stopMusic()
                    makeMusic("sounds/flag_runner.wav")
                    playMusic(loops=100)
                    flag_in_play = True

            #touched a player - helping or tagging?
            elif type(thing)==type(player): 

                #helping
                if player.team==thing.team:
                    playSound(revived_sound)
                    thing.energy = 10
                    player.energy = 10

                #tagging - pt on opponent side  
                elif thing.has_flag and thing.energy>0:
                    playSound(dropped_flag_sound)
                    thing.energy = 0

                    thing.sprite = thing.incapacitated_sprite
                    showSprite(player.sprite)

                    #drop the flag
                    if thing.team=='blue':
                    #TODO make sure it isn't dropped off the map
                        moveSprite(blue_flag, thing.x + 40, thing.y + 40)
                        showSprite(blue_flag)
                    else:
                        moveSprite(red_flag, thing.x + 40, thing.y + 40)
                        showSprite(red_flag)

                #tagging - pt on opponent side  
                elif thing.on_opponent_side and thing.energy>0:
                    playSound(tagged_sound)
                    thing.energy = 0

In [4]:
%tb
#display icon, title, background
#add character, flag, flag area, speed and terrain check

pygame.init()
config = Config()

speed_terr = {v:k for k,v in config.terrain_speeds.items()}

pyg.screenSize(840, 640)
pyg.setBackgroundImage(config.map_default_path)
pyg.setIcon(config.game_icon_image_path)
pyg.setWindowTitle(config.game_name)

speed_arr = np.load(config.map_default_speed_array)

blue_player_sprite = pyg.makeSprite(config.blue_player_sprite_path, 32)
blue_player_x_pos = random.randint(config.map_border_size, config.screen_width//3)
blue_player_y_pos = random.randint(config.map_border_size, config.screen_height - config.map_border_size)
c_idx = math.floor((blue_player_x_pos - config.map_border_size)//config.terrain_tile_size)
r_idx = math.floor((blue_player_y_pos - config.map_border_size)//config.terrain_tile_size)

speed = int(speed_arr[r_idx, c_idx])
print('player is on terrain %s with speed %d' % (speed_terr[speed], speed))

pyg.moveSprite(blue_player_sprite, x=blue_player_x_pos, y=blue_player_y_pos, centre=True)
pyg.showSprite(blue_player_sprite)

blue_flag_sprite = pyg.makeSprite(config.blue_flag_sprite_path)
blue_flag_x_pos = random.randint(config.map_border_size+50, config.screen_width//3)
blue_flag_y_pos = random.randint(config.map_border_size+50, config.screen_height - config.map_border_size)
pyg.drawEllipse(blue_flag_x_pos, blue_flag_y_pos, 100, 100, colour='blue', linewidth=3)
pyg.moveSprite(blue_flag_sprite, x=blue_flag_x_pos, y=blue_flag_y_pos, centre=True)
pyg.showSprite(blue_flag_sprite)

pyg.tick(10)
    
pyg.endWait()

No traceback available to show.


player is on terrain swamp with speed 7
Press ESC to quit


In [6]:
#display icon, title, background, one sprite
pygame.init()
config = Config()

pyg.screenSize(840, 640)
pyg.setBackgroundImage('../app/resources/maps/map_840_640.png')
pyg.setIcon(config.game_icon_image_path)
pyg.setWindowTitle(config.game_name)

blue_flag_sprite = pyg.makeSprite('../app/resources/sprites/blue_flag_20.png')
blue_flag_x_pos = random.randint(config.map_border_size+50, config.screen_width//3)
blue_flag_y_pos = random.randint(config.map_border_size+50, config.screen_height - config.map_border_size)
print(blue_flag_x_pos, blue_flag_y_pos)
pyg.moveSprite(blue_flag_sprite, x=200, y=200)
pyg.showSprite(blue_flag_sprite)

pyg.endWait()

87 410
Press ESC to quit


In [4]:
#allowed movement

speed = int(tile_speeds[player_r_idx, player_c_idx])
speed_terr = {v:k for k,v in config.terrain_speeds.items()}
print('player is on terrain %s with speed %d' % (speed_terr[speed], speed))

prev_blue_player_x_pos, prev_blue_player_y_pos, prev_speed = blue_player_x_pos, blue_player_y_pos, speed

nextFrame = pyg.clock()
frame = 0
while True:
    if pyg.clock() > nextFrame:                         # We only animate our character every 80ms.
        frame = (frame+1)%8                         # There are 8 frames of animation in each direction
        nextFrame += 80                             # so the modulus 8 allows it to loop
    
    if pyg.keyPressed("d"):
        pyg.changeSpriteImage(blue_player_sprite, 0*8+frame)    # 0*8 because right animations are the 0th set in the sprite sheet
        blue_player_x_pos += speed

    elif pyg.keyPressed("s"):
        pyg.changeSpriteImage(blue_player_sprite, 1*8+frame)    # down facing animations are the 1st set
        blue_player_y_pos += speed

    elif pyg.keyPressed("a"):
        pyg.changeSpriteImage(blue_player_sprite, 2*8+frame)    # and so on
        blue_player_x_pos -= speed

    elif pyg.keyPressed("w"):
        pyg.changeSpriteImage(blue_player_sprite, 3*8+frame)
        blue_player_y_pos -= speed

    else:
        pyg.changeSpriteImage(blue_player_sprite, 1 * 8 + 5)  # the static facing front look
        
    #don't allow player on the borders
    if blue_player_x_pos<=config.map_border_size or \
    blue_player_x_pos>=config.screen_width - config.map_border_size or \
    blue_player_y_pos<=config.map_border_size or \
    blue_player_y_pos>=config.screen_height - config.map_border_size:
        speed = 0
    else:
        #what terrain is the player on now?
        c_idx = math.floor((blue_player_x_pos-config.map_border_size)//config.terrain_tile_size)
        r_idx = math.floor((blue_player_y_pos-config.map_border_size)//config.terrain_tile_size)
        speed = int(speed_arr[r_idx, c_idx])
    
    if not speed==prev_speed:
        print('player moved from %s to %s speed %d to %d.' % (speed_terr[prev_speed], speed_terr[speed], prev_speed, speed))

    #if player tried to walk where they're not allowed, revert to prev pos & speed or else
    # they won't be able to leave it
    if not speed:
        blue_player_x_pos, blue_player_y_pos, speed = prev_blue_player_x_pos, prev_blue_player_y_pos, prev_speed
    else:
        pyg.moveSprite(blue_player_sprite, blue_player_x_pos, blue_player_y_pos)
        prev_blue_player_x_pos, prev_blue_player_y_pos, prev_speed = blue_player_x_pos, blue_player_y_pos, speed
    
    pyg.tick(10)
    
pyg.endWait()

No traceback available to show.


flag on r=1, c=1, x=70, y=70
removed close flag area 0 1
removed close flag area 5 2
removed close flag area 0 2
removed close flag area 4 1
removed close flag area 5 1
removed close flag area 0 0
removed close flag area 0 5
removed close flag area 5 0
removed close flag area 2 0
removed close flag area 1 0
removed close flag area 2 1
removed close flag area 4 0
removed close flag area 3 0
removed close flag area 0 4
player on r=4, c=7, x=160, y=100
player is on terrain swamp with speed 7
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to plain speed 7 to 10

player moved from plain to swamp speed 10 to 7.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to plain speed 7 to 10.
player moved from plain to swamp speed 10 to 7.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to l

player moved from swamp to plain speed 7 to 10.
player moved from plain to swamp speed 10 to 7.
player moved from swamp to plain speed 7 to 10.
player moved from plain to swamp speed 10 to 7.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to

SystemExit: 

c:\ml\env\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# some game logic

#sounds
#victory magic
grabbed_flag_sound = makeSound("sounds/grabbed_flag.wav")
#bonk, tweet tweet
incapacitated_sound = makeSound('sounds/incapacitated.wav')
#floop
dropped_flag_sound = makeSound('sounds/dropped_flag.wav')
#slap
tagged_sound = makeSound('sounds/tagged.wav')
#bling
revived_sound = makeSound('sounds/revived.wav')

#music
makeMusic("sounds/default.wav")
playMusic(loops=100)


#sprites - flags, flag areas, territory overlays
blue_flag = makeSprite('blue_flag.png')
red_flag = makeSprite('red_flag.png')


#human_player if not observe mode - players wrap sprites for: normal, has flag, is incapacitated, tagging
players = [HumanPlayer('blue', 0)]
players.extend([AgentPlayer('blue', i+1) for i in team_size-1])
players.extend([AgentPlayer('red', i) for i in team_size])


flag_in_play = True

game_winner=''

while True:
    if game_winner:
        #hide all sprites
        hideAll()
        if game_winner=='blue':
            setBackgroundImage(blue_wins_background)
            stopMusic()
            makeMusic("sounds/blue_victory.wav")
            playMusic(loops=1)
        else:
            setBackgroundImage(red_wins_background)
            stopMusic()
            makeMusic("sounds/red_victory.wav")
            playMusic(loops=1)
        pause(5000)
        break
        
    #quit?
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    
    #process players
    for player in players:

        #what is the player touching
        player_touching = allTouching(player.sprite)
        if player_touching:
            for thing in player_touching:
                #how check type?
                
                #touched flag area and has flag - won
                if player.has_flag and type(thing)==type(blue_flag_area) and player.team=='blue' \
                or type(thing)==type(red_flag_area) and player.team=='red': 
                    game_winner=player.team
                    break
                    
                #touched a flag
                if type(thing)==type(red_flag) and player.team=='blue' or type(thing)==type(blue_flag) and player.team=='red': 
                    #play sound
                    playSound(grabbed_flag_sound)

                    #remove flag
                    if player.team=='blue'
                        killSprite(red_flag)
                    else:
                        killSprite(blue_flag)

                    #remove player without flag
                    killSprite(player.sprite)

                    #draw player with flag
                    player.sprite = player.has_flag_sprite
                    showSprite(player.sprite)

                    #if a flag not already in play, play tense music
                    if not flag_in_play:
                        stopMusic()
                        makeMusic("sounds/flag_runner.wav")
                        playMusic(loops=100)
                        flag_in_play = True
                
                #touched a player - helping or tagging?
                elif type(thing)==type(player): 
                    
                    #helping
                    if player.team==thing.team:
                        playSound(revived_sound)
                        thing.energy = 10
                        player.energy = 10
                    
                    #tagging - pt on opponent side  
                    elif thing.has_flag and thing.energy>0:
                        playSound(dropped_flag_sound)
                        thing.energy = 0
                        
                        thing.sprite = thing.incapacitated_sprite
                        showSprite(player.sprite)
            
                        #drop the flag
                        if thing.team=='blue':
                        #TODO make sure it isn't dropped off the map
                            moveSprite(blue_flag, thing.x + 40, thing.y + 40)
                            showSprite(blue_flag)
                        else:
                            moveSprite(red_flag, thing.x + 40, thing.y + 40)
                            showSprite(red_flag)
                            
                    #tagging - pt on opponent side  
                    elif thing.on_opponent_side and thing.energy>0:
                        playSound(tagged_sound)
                        thing.energy = 0

In [ ]:
pygame.init()
config = Config()

try:
    pyg.screenSize(config.screen_width, config.screen_height)
    pyg.setBackgroundImage(config.map_default_path)
    pyg.setIcon(config.game_icon_image_path)
    pyg.setWindowTitle(config.game_name)

    border = config.map_border_size
    tile_size = config.terrain_tile_size
    terr_speeds = config.terrain_speeds

    speed_arr = np.load(config.map_default_speed_array)

    blueSprite  = pyg.makeSprite(config.blue_player_sprite_path, 32)  # links.gif contains 32 separate frames of animation.

    x_pos = random.randint(border,config.screen_width//3)
    y_pos = random.randint(border,config.screen_height-border)

    c_idx = math.floor((x_pos-border)//tile_size)
    r_idx = math.floor((y_pos-border)//tile_size)

    speed = int(speed_arr[r_idx, c_idx])
    speed_terr = {v:k for k,v in terr_speeds.items()}
    print(x_idx, y_idx, speed, speed_terr[speed])

    pyg.moveSprite(blueSprite, x=x_pos, y=y_pos, centre=True)
    pyg.showSprite(blueSprite)

    nextFrame = pyg.clock()
    frame = 0
    i=0
    while True:
        i+=1
        if i>1000:
            print('times up')
            break

        if pyg.clock() > nextFrame:                         # We only animate our character every 80ms.
            frame = (frame+1)%8                         # There are 8 frames of animation in each direction
            nextFrame += 80                             # so the modulus 8 allows it to loop

        c_idx = math.floor((x_pos-border)/tile_size)
        r_idx = math.floor((y_pos-border)/tile_size)
        #print(x_pos, y_pos, x_idx, y_idx)
        speed = speed_arr[r_idx, c_idx]

        if pyg.keyPressed("d"):
            pyg.changeSpriteImage(blueSprite, 0*8+frame)    # 0*8 because right animations are the 0th set in the sprite sheet
            x_pos += speed

        elif pyg.keyPressed("s"):
            pyg.changeSpriteImage(blueSprite, 1*8+frame)    # down facing animations are the 1st set
            y_pos += speed

        elif pyg.keyPressed("a"):
            pyg.changeSpriteImage(blueSprite, 2*8+frame)    # and so on
            x_pos -= speed

        elif pyg.keyPressed("w"):
            pyg.changeSpriteImage(blueSprite, 3*8+frame)
            y_pos -= speed

        else:
            pyg.changeSpriteImage(blueSprite, 1 * 8 + 5)  # the static facing front look

        pyg.moveSprite(blueSprite, x_pos, y_pos)

        pyg.tick(120)
except Exception as ex:
    print(ex)
    pyg.killSprite(blueSprite)
    pyg.endWait()

2 15 10 plain


In [ ]:
#untested code skeleton
#class for menu GUI elements

class Menu():
    def __init__(self, config, map_generator):
        self.config = config
        
        #holds a generated map, can replace generated map with .generate_map()
        self.map_generator = map_generator
        
        #the dict of values populated from the menu
        self.menu_values = {}
        
        self.menu_background = pyg.makeSprite(config.menu_background_image_path)
        pyg.moveSprite(self.menu_background, 200,150) #TODO - location
        pyg.showSprite(self.menu_background)
            
        self.text_label = pyg.makeLabel("Capture the Flag", 22, 200, 50, "white", font="impact")#TODO - location
        pyg.showLabel(self.text_label)
        
        #elements of the menu
        self.game_mode_select = pyg.makeSprite(config.menu_game_mode_select_image_path)
        pyg.moveSprite(self.game_mode_select, 200,150)#TODO - location
        pyg.showSprite(self.game_mode_select)
        
        self.team_size_select = pyg.makeSprite(config.menu_team_size_select_image_path)
        pyg.moveSprite(self.team_size_select, 200,150)#TODO - location
        pyg.showSprite(self.team_size_select)
            
        self.regenerate_map_button = pyg.makeSprite(config.menu_regenerate_map_button_image_path)
        pyg.moveSprite(self.regenerate_map_button, 200,150)#TODO - location
        pyg.showSprite(self.regenerate_map_button)
            
        self.start_game_button = pyg.makeSprite(config.menu_start_game_button_image_path)
        pyg.moveSprite(self.start_game_button, 200,150)#TODO - location
        pyg.showSprite(self.start_game_button)
            
        #display small version of generated map so user can choose to regenerate it
        #TODO use the menu_image in map_generator to paste on the sprite
        self.map_display = pyg.makeSprite(config.menu_map_display_image_path)
        pyg.moveSprite(self.map_display, 200,150)#TODO - location
        pyg.showSprite(self.map_display)
        
    
    def get_values(self):
        '''When Start Game button is pressed, returns dict of values populated from the menu.'''
        game_mode = self.config.game_mode_play
        team_size = self.config.team1_size
        player_team = self.config.team1_name
        
        while True:
            #display menu for choosing game mode, number of players, team, map
            if pyg.spriteClicked(self.game_mode_select):
                self.menu_values['game_mode'] = self.get_game_mode()
                
            elif pyg.spriteClicked(self.team_size_select):
                self.menu_values['team_size'] = self.get_team_size()
                
            elif pyg.spriteClicked(self.player_team_select):
                self.menu_values['player_team'] = self.get_player_team()
                
            elif pyg.spriteClicked(self.regenerate_map_button):
                self.map_generator.generate_map()
                self.update_map(self.map_generator.menu_image)
                
            elif pyg.spriteClicked(self.start_game_button):
                #destroy the menu and return the values
                pyg.hideLabel(self.text_label)
                pyg.killSprite(self.game_mode_select)
                pyg.killSprite(self.team_size_select)
                pyg.killSprite(self.regenerate_map_button)
                pyg.killSprite(self.start_game_button)
                pyg.killSprite(self.map_display)
                pyg.killSprite(self.menu_background)
                return self.menu_values
            
            tick(config.menu_frame_rate)
            
                
    def get_game_mode(self):
        #TODO get from selection
        return self.config.game_mode_play
    
    def get_team_size(self):
        #TODO get from selection
        return self.config.team1_size
    
    def get_player_team(self):
        #TODO get from selection
        return self.config.team1_name
    
    def update_map(self, im):
        #TODO update map image sprite
        pass

In [ ]:
#untested code skeleton
# class for game status window next to map

class GameStatusWindow():
    def __init__(self):
        pass
    def display(self):
        pass

In [ ]:
#untested code skeleton
#class for map GUI elements

class GameMapWindow():
    def __init__(self, game_status_window):
        pass
    def display(self):
        pass

In [ ]:
#untested code skeleton
#top class for creating all GUI elements and running the game

class Game():
    def __init__(self, config):
        self.config = config
        
        #instantiate pygame objects using game vars from config
        pyg.screenSize(config.screen_width, config.screen_height)
        
        pyg.setIcon(config.game_icon_image_path)
        pyg.setWindowTitle(config.game_name)
        pyg.setBackgroundImage(config.game_background_image_path)
        
        # menu
        
        #map may be regenerated in menu, so access map_generator through menu object
        map_generator = MapGenerator()
        map_generator.generate_map()
        
        self.menu = Menu(config, map_generator)
        
        
    def play(self):
        #main game loop
        while True:
            #wait for values from the menu - currently includes game_mode, team_size, player_team
            menu_values = self.menu.get_values()
            game_mode = menu_values['game_mode']
            team_size = menu_values['team_size']
            player_team = menu_values['player_team']
            
            #the generated map is accessible through the map generator in menu
            #update background with generated map
            #self.menu.map_generator
            
            #create player and agents
            
            #update sprites according to their actions
            
            #check for win

            pyg.tick(config.time_delay)
            
        pyg.endWait()

In [ ]:
config = Config()
game = Game(config)
game.play()